<h1>VIT Testing - tamncheese Jason Kahei Tam<h1>


Import

In [ ]:
import torch
import torch.nn as nn
import os
from torch.utils.data import DataLoader, Dataset
from transformers import ViTImageProcessor, ViTForImageClassification, AutoFeatureExtractor, AutoModel, \
    AutoModelForImageClassification, AutoImageProcessor
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from torchvision import datasets, transforms
import pandas as pd


C:\Users\jason\Documents\OMSCS\DSGT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU or CUDA


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Prepare Dataset

In [ ]:
file_list = []
for root, dirs, files, in os.walk("training"):
    for file in files:
        file_list.append(file)
file_list

train_df = pd.read_csv("training.csv")

In [ ]:


class FungiDataset(Dataset):
    def __init__(self, df, extractor, img_dir, transform = None):
        self.df = df
        self.transform = transform
        self.img_dir = img_dir
        self.extractor = extractor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]["image_path"])
        label = self.df.iloc[idx]["poisonous"]
        # print(img_path)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        image = self.extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)
        return image, torch.tensor(label, dtype = torch.long)



Configure DINOv2


In [ ]:
extractor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModelForImageClassification.from_pretrained('facebook/dinov2-base', num_labels=2)

Load Data


In [ ]:
img_dir = "training"
train_dataset = FungiDataset(train_df, extractor, img_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

Train the Model


In [1]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)
criterion = nn.CrossEntropyLoss()

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss} for epoch {epoch + 1}/{epochs}")



NameError: name 'torch' is not defined